In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.llamma import LLAMMA

In [3]:
class Oracle():
    def __init__(self):
        pass
    def price(self):
        return 1490

llamma = LLAMMA(100, 1500, Oracle(), 0)